# Notebook to house some utility functions for Mujoco modeling

### Imports

In [1]:
import os

import numpy as np

from scipy.spatial.transform import Rotation as R

### Helper functions

In [2]:
def my_norm(x):
    """
    Get vector norm
    """
    return np.sqrt(np.sum(np.asarray(x)**2))

In [3]:
def unit_axis_angle(a, b):
    """ 
    Get the rotation vector between two vectors a and b 
    in axis, angle format
        
    """
    # get unit vectors 
    a_hat = a/my_norm(a)
    b_hat = b/my_norm(b)
    
    # cross product defines axis of rotation
    n = np.cross(a_hat, b_hat)
    n_hat = n/my_norm(n)
    
    # get rotation angle from arc cosine of dot product
    theta = np.arccos(np.dot(a_hat, b_hat))
    
    return n_hat, theta

In [4]:
def rotation_matrix(axis, angle):
    """
    Convert axis-angle representation to a rotation matrix
    
    """
    ax, ay, az = axis[0], axis[1], axis[2]
    s = np.sin(angle)
    c = np.cos(angle)
    u = 1 - c
    return ( ( ax*ax*u + c,    ax*ay*u - az*s, ax*az*u + ay*s ),
             ( ay*ax*u + az*s, ay*ay*u + c,    ay*az*u - ax*s ),
             ( az*ax*u - ay*s, az*ay*u + ax*s, az*az*u + c    ) )


In [5]:
def euler_to_axis(start_euler, end_euler, seq='XYZ', deg_flag=False):
    """
    Function for a specialized problem: given two sets of euler angles 
    specifying the orientation of two bodies, what is the (joint) axis
    that we should rotate the "start" body by to get it into "end" 
    orientation?
    
    from scipy.spatial.transform import Rotation as R
    """
    # get rotation matrices from euler angle representations
    rot_mat_start = R.from_euler(seq, start_euler,degrees=deg_flag).as_matrix()
    rot_mat_end = R.from_euler(seq, end_euler,degrees=deg_flag).as_matrix()
    
    # get rotation from start to end orientation
    rot_mat = rot_mat_end.dot(np.transpose(rot_mat_start))

    # get axis angle from combined rotation matrix
    r = R.from_matrix(rot_mat).as_rotvec()
    axis = r/my_norm(r)
    angle = my_norm(r)
    if deg_flag:
        angle = np.rad2deg(angle)

    return axis, angle


### Tests

In [6]:
# quick test of finding rotation matrix between two vectors
a = np.random.rand(3)
b = np.random.rand(3) 

s = my_norm(a) / my_norm(b)
b = s*b

axis, angle = unit_axis_angle(a, b)
rot_mat = rotation_matrix(axis, angle)
c = rot_mat @ a

print('a = ', a)
print('b = ', b)
print('c = ', c)

a =  [0.96375385 0.13820297 0.69180582]
b =  [0.94152245 0.55324943 0.48370173]
c =  [0.94152245 0.55324943 0.48370173]


### Getting slide props for SLA

In [7]:
# get center of mass for SLA in both positions
sla_up_cm = np.array([-0.31764, -2.65612, 9.31039])
sla_down_cm = np.array([-0.701694, -2.84062, 9.40747])

# so direction of translation vector is 
translation_vec = sla_down_cm - sla_up_cm 
t_vec_hat = translation_vec/my_norm(translation_vec)

print(my_norm(translation_vec))

0.43699227832537274


In [8]:
# get rotation between global x axis and rotation vec
x_hat = np.array([1,0,0])
axis, angle = unit_axis_angle(x_hat, t_vec_hat)
rot_mat = rotation_matrix(axis, angle)

print(rot_mat)

((-0.8788576344455308, 0.42220425657641963, -0.22215495516769035), (-0.42220425657641963, -0.47146238605599833, 0.7742524034597099), (0.22215495516769035, 0.7742524034597099, 0.5926047516104676))


In [9]:
# convert to rotation object and express as euler angles 
r0 = R.from_matrix(rot_mat)
r1 = R.as_euler(r0, 'XYZ')

print(np.rad2deg(r1))

[ -52.56993181  -12.83563548 -154.3403148 ]


### Getting hinge props for SLA

In [10]:
# Now get axis angle representation of rotation between the two objects, according to their euler angles 
sla_up_euler = [-0.000003,-4.3418,-0.000001]
sla_down_euler = [-12.7807, 5.71, 10.2]  # NB: this reflects the difference in mesh  # [-0.000002,9.88261,0.000001]

# covnert to radian
sla_up_euler = [np.deg2rad(d) for d in sla_up_euler]
sla_down_euler = [np.deg2rad(d) for d in sla_down_euler]

# get axis angle
sla_axis, sla_angle = euler_to_axis(sla_up_euler, sla_down_euler)

print(sla_axis)
print(np.rad2deg(sla_angle))

[-0.65983511  0.5828032   0.47429744]
19.08376565672737


In [11]:
# a little indirect, but not get the euler angles needed to make my empty axis object have the right orientation
x_hat = np.array([1,0,0])
to_rot_axis, to_rot_angle = unit_axis_angle(x_hat, sla_axis)
to_rot_mat = rotation_matrix(to_rot_axis, to_rot_angle)

hinge_euler = (R.from_matrix(to_rot_mat)).as_euler('XYZ', degrees=True)

print(hinge_euler)

[ 67.37466597 -28.31361635 138.54726142]


### Get Euler angles that will align xhat to a given unit vector

In [13]:
x_hat = np.array([1,0,0])
rot_axis = np.array([0.5851859757164767, -0.74410596040849, -0.3222789063983468])
to_rot_axis, to_rot_angle = unit_axis_angle(x_hat, rot_axis)
to_rot_mat = rotation_matrix(to_rot_axis, to_rot_angle)

hinge_euler = (R.from_matrix(to_rot_mat)).as_euler('XYZ', degrees=True)
print(hinge_euler)

[  9.19577105  18.80079961 -51.81748813]


## Scratch

In [12]:
time_pts = np.linspace(1.75, 2.75, 11)
print(time_pts)

[1.75 1.85 1.95 2.05 2.15 2.25 2.35 2.45 2.55 2.65 2.75]
